In [2]:
!ngrok authtoken 2obZ8eR47JKtaorKnBqEn5KkHlS_5Kk68KWttZanGGY618BRu


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
from flask import Flask, render_template, request, send_file
import torch
from transformers import MarianMTModel, MarianTokenizer
import speech_recognition as sr
from pydub import AudioSegment
from gtts import gTTS
import os
from pyngrok import ngrok

app = Flask(__name__)

# Load the MarianMT model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def convert_to_wav(file_path):
    """Convert audio file to .wav format using pydub."""
    audio = AudioSegment.from_file(file_path)
    wav_path = "temp_audio.wav"
    audio.export(wav_path, format="wav")
    return wav_path

def translate_text(user_input):
    """Translate English text to Spanish with full sentence handling."""
    sentences = user_input.split(". ")
    translated_sentences = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            translated_tokens = model.generate(**inputs)
        translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        translated_sentences.append(translated_sentence)
    return ". ".join(translated_sentences)

def text_to_speech(text, filename="translated_audio.mp3"):
    """Convert text to speech in Spanish."""
    tts = gTTS(text=text, lang='es')
    tts.save(filename)
    return filename

def translate_audio_file(file_path):
    """Detect English text from an audio file and translate it."""
    recognizer = sr.Recognizer()
    if not file_path.endswith(".wav"):
        file_path = convert_to_wav(file_path)

    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    try:
        user_input = recognizer.recognize_google(audio)
        return user_input
    except (sr.UnknownValueError, sr.RequestError):
        return None

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        input_type = request.form.get("input_type")
        output_format = request.form.get("output_format")
        translation_result = None
        translated_audio_path = None
        translated_file_path = None

        # Handling Text Input
        if input_type == "text":
            user_input = request.form["text_input"]
            translation_result = translate_text(user_input)

            # If output as text file
            if output_format == "file":
                translated_file_path = "translated_text.txt"
                with open(translated_file_path, "w") as f:
                    f.write(translation_result)

            # If output as audio file
            elif output_format == "audio":
                translated_audio_path = text_to_speech(translation_result)

        # Handling File Input
        elif input_type == "file":
            text_file = request.files["text_file"]
            user_input = text_file.read().decode("utf-8")
            translation_result = translate_text(user_input)

            if output_format == "file":
                translated_file_path = "translated_text.txt"
                with open(translated_file_path, "w") as f:
                    f.write(translation_result)
            elif output_format == "audio":
                translated_audio_path = text_to_speech(translation_result)

        # Handling Audio Input
        elif input_type == "audio":
            audio_file = request.files["audio_file"]
            audio_file_path = "uploaded_audio.wav"
            audio_file.save(audio_file_path)

            user_input = translate_audio_file(audio_file_path)
            if user_input:
                translation_result = translate_text(user_input)

                if output_format == "file":
                    translated_file_path = "translated_text.txt"
                    with open(translated_file_path, "w") as f:
                        f.write(translation_result)
                elif output_format == "audio":
                    translated_audio_path = text_to_speech(translation_result)

        return render_template("index.html",
                               translation=translation_result,
                               audio_file=translated_audio_path,
                               text_file=translated_file_path)
    return render_template("index.html")

@app.route("/download_audio")
def download_audio():
    return send_file("translated_audio.mp3", as_attachment=True)

@app.route("/download_text")
def download_text():
    return send_file("translated_text.txt", as_attachment=True)

# Start ngrok and launch the app
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))
app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://d558-34-66-83-27.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 08:42:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 08:42:56] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 08:43:10] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 08:43:26] "POST / HTTP/1.1" 200 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/speech_recognition/__init__.py", line 238, in __enter__
    self.audio_reader = wave.open(self.filename_or_fileobject, "rb")
  File "/usr/lib/python3.10/wave.py", line 509, in open
    return Wave_read(f)
  File "/usr/lib/python3.10/wave.py", line 163, in __init__
    self.initfp(f)
  File "/usr/lib/python3.10/wave.py", line 130, in initfp
    raise Error('file doe

In [18]:
!mkdir -p templates
with open("templates/index.html", "w") as f:
    f.write('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>English to Spanish Translator</title>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap" rel="stylesheet">
    <style>
        /* Reset */
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
            font-family: 'Roboto', sans-serif;
        }
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            background: linear-gradient(135deg, #74ebd5, #acb6e5);
            color: #333;
        }
        .container {
            background-color: #ffffff;
            padding: 2em;
            border-radius: 12px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            width: 100%;
            text-align: center;
        }
        h1 {
            font-weight: 700;
            margin-bottom: 1em;
            color: #4a4a4a;
        }
        .section-title {
            font-size: 1.2em;
            font-weight: 500;
            color: #555;
            margin-top: 1.5em;
        }
        label, button {
            display: block;
            width: 100%;
            margin-top: 0.8em;
            text-align: left;
            color: #333;
        }
        input[type="radio"] {
            margin-right: 0.5em;
        }
        textarea, input[type="file"], select {
            width: 100%;
            padding: 0.8em;
            margin-top: 0.5em;
            border: 1px solid #ccc;
            border-radius: 8px;
            font-size: 1em;
        }
        button {
            margin-top: 1.5em;
            padding: 12px 24px;
            background-color: #4CAF50;
            color: #fff;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-weight: 500;
            transition: background-color 0.3s ease;
        }
        button:hover {
            background-color: #45a049;
        }
        .output-container {
            margin-top: 2em;
            padding: 1em;
            background-color: #f9f9f9;
            border-radius: 8px;
            border: 1px solid #eee;
        }
        .output-title {
            font-size: 1.1em;
            font-weight: 500;
            margin-bottom: 1em;
        }
        .download-link {
            display: inline-block;
            margin-top: 1em;
            color: #4CAF50;
            font-weight: 500;
            text-decoration: none;
            border-bottom: 1px solid transparent;
            transition: border-bottom 0.3s ease;
        }
        .download-link:hover {
            border-bottom: 1px solid #4CAF50;
        }
        .toggle-group {
            display: flex;
            justify-content: space-around;
            margin-top: 1em;
        }
        .toggle-group label {
            cursor: pointer;
            flex: 1;
            text-align: center;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 8px;
            background-color: #f4f4f4;
            transition: background-color 0.3s ease;
        }
        .toggle-group label:hover, .toggle-group label.active {
            background-color: #e0f7e9;
        }
        input[type="radio"]:checked + label {
            background-color: #d4f1d8;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>English to Spanish Translator</h1>
        <form method="POST" enctype="multipart/form-data">
            <div class="section-title">Choose input type:</div>
            <div class="toggle-group">
                <input type="radio" id="text" name="input_type" value="text" checked hidden>
                <label for="text">Text</label>

                <input type="radio" id="audio" name="input_type" value="audio" hidden>
                <label for="audio">Audio</label>

                <input type="radio" id="file" name="input_type" value="file" hidden>
                <label for="file">Text File</label>
            </div>

            <div id="text-input" style="display: block;">
                <label for="text_input">Enter text:</label>
                <textarea name="text_input" id="text_input" rows="4" placeholder="Type the text to translate..."></textarea>
            </div>

            <div id="audio-input" style="display: none;">
                <label for="audio_file">Upload audio file:</label>
                <input type="file" name="audio_file" id="audio_file" accept=".wav, .mp3, .m4a">
            </div>

            <div id="file-input" style="display: none;">
                <label for="text_file">Upload text file:</label>
                <input type="file" name="text_file" id="text_file" accept=".txt">
            </div>

            <div class="section-title">Select output format:</div>
            <div class="toggle-group">
                <input type="radio" id="output_text" name="output_format" value="text" checked hidden>
                <label for="output_text">Text</label>

                <input type="radio" id="output_audio" name="output_format" value="audio" hidden>
                <label for="output_audio">Audio</label>

                <input type="radio" id="output_file" name="output_format" value="file" hidden>
                <label for="output_file">Text File</label>
            </div>

            <button type="submit">Translate</button>
        </form>

        {% if translation %}
            <div class="output-container">
                <div class="output-title">Translation Result:</div>
                <p>{{ translation }}</p>
            </div>
        {% endif %}

        {% if audio_file %}
            <div class="output-container">
                <div class="output-title">Audio Translation:</div>
                <audio controls>
                    <source src="{{ url_for('download_audio') }}" type="audio/mp3">
                    Your browser does not support the audio element.
                </audio>
            </div>
        {% endif %}

        {% if text_file %}
            <div class="output-container">
                <a href="{{ url_for('download_text') }}" class="download-link" download>Download Translated Text File</a>
            </div>
        {% endif %}
    </div>

    <script>
        document.querySelectorAll('.toggle-group input[type="radio"]').forEach(radio => {
            radio.addEventListener('change', function() {
                document.getElementById('text-input').style.display = radio.value === 'text' ? 'block' : 'none';
                document.getElementById('audio-input').style.display = radio.value === 'audio' ? 'block' : 'none';
                document.getElementById('file-input').style.display = radio.value === 'file' ? 'block' : 'none';
            });
        });

        document.querySelectorAll('.toggle-group label').forEach(label => {
            label.addEventListener('click', function() {
                document.querySelectorAll('.toggle-group label').forEach(l => l.classList.remove('active'));
                label.classList.add('active');
            });
        });
    </script>
</body>
</html>

    ''')
